In [ ]:
# Instalar bibliotecas necessárias (apenas na primeira execução)
!pip install -U spacy gensim
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

# Baixar recursos necessários
nltk.download("stopwords")
nltk.download("rslp")

# Inicializar modelos
nlp = spacy.load("pt_core_news_sm")
stopwords_pt = set(stopwords.words("portuguese"))
stemmer = RSLPStemmer()

print("Imports concluídos!")

In [ ]:
import io
import re
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd

import spacy
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

from gensim.models import Word2Vec

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from scipy.sparse import csr_matrix

# Carrega modelo de linguagem em PT-BR para lematização
nlp = spacy.load("pt_core_news_sm")

# Stopwords em português
stopwords_pt = set(stopwords.words("portuguese"))

# Stemmer (Português)
stemmer = RSLPStemmer()

print("Imports concluídos.")

In [4]:
from google.colab import files
uploaded = files.upload()

# Espera que o arquivo se chame exatamente 'buscape.csv'
df = pd.read_csv(io.BytesIO(uploaded['buscape.csv']))

df.head()


Saving buscape.csv to buscape.csv


,original_index,review_text,review_text_processed,review_text_tokenized,polarity,rating,kfold_polarity,kfold_rating
0,4_55516,"Estou muito satisfeito, o visor é melhor do qu...","estou muito satisfeito, o visor e melhor do qu...","['estou', 'muito', 'satisfeito', 'visor', 'mel...",1.0,4,1,1
1,minus_1_105339,"""muito boa\n\nO que gostei: preco\n\nO que não...","""muito boa\n\no que gostei: preco\n\no que nao...","['muito', 'boa', 'que', 'gostei', 'preco', 'qu...",1.0,5,1,1
2,23_382139,"Rápida, ótima qualidade de impressão e fácil d...","rapida, otima qualidade de impressao e facil d...","['rapida', 'otima', 'qualidade', 'de', 'impres...",1.0,5,1,1
3,2_446456,Produto de ótima qualidade em todos os quesito!,produto de otima qualidade em todos os quesito!,"['produto', 'de', 'otima', 'qualidade', 'em', ...",1.0,5,1,1
4,0_11324,Precisava comprar uma tv compatível com meu dv...,precisava comprar uma tv compativel com meu dv...,"['precisava', 'comprar', 'uma', 'tv', 'compati...",1.0,5,1,1


Visão geral do dataset

In [5]:
print("Shape do dataset:", df.shape)
print("\nColunas disponíveis:", df.columns.tolist())

print("\nDistribuição da coluna 'polarity':")
print(df["polarity"].value_counts(normalize=True))

df[["review_text", "polarity"]].head(10)


Shape do dataset: (84991, 8)

Colunas disponíveis: ['original_index', 'review_text', 'review_text_processed', 'review_text_tokenized', 'polarity', 'rating', 'kfold_polarity', 'kfold_rating']

Distribuição da coluna 'polarity':
polarity
1.0    0.907507
0.0    0.092493
Name: proportion, dtype: float64


,review_text,polarity
0,"Estou muito satisfeito, o visor é melhor do qu...",1.0
1,"""muito boa\n\nO que gostei: preco\n\nO que não...",1.0
2,"Rápida, ótima qualidade de impressão e fácil d...",1.0
3,Produto de ótima qualidade em todos os quesito!,1.0
4,Precisava comprar uma tv compatível com meu dv...,1.0
5,"eu adorei este secador é muito bom,potente e d...",1.0
6,bom\n\nO que gostei: muitos aplicativos gratis...,NaN
7,positiva\n\nO que gostei: cumpriu com as espec...,1.0
8,"Muito satisfeita com o telefone, atendeu as mi...",1.0
9,Estou muito contente com a utlização da maquin...,1.0


# Questão 1 – Pré-processamento (`preprocessing`)

Nesta etapa, serão implementadas funções de manipulação e limpeza do texto, incluindo:

- Conversão para minúsculas;
- Remoção de URLs;
- Remoção de menções (`@usuario`);
- Remoção de hashtags (`#tag`);
- Remoção de caracteres não alfabéticos;
- Tokenização;
- Remoção de stopwords;
- Lematização (usando spaCy);
- Stemming (usando RSLPStemmer).

O objetivo é preparar o texto para as etapas de extração de atributos e classificação.
As funções são organizadas de forma que possam ser reutilizadas nas outras questões.


In [6]:
# Regex úteis
URL_RE = re.compile(r"http\S+|www\.\S+")
MENTION_RE = re.compile(r"@\w+")
HASHTAG_RE = re.compile(r"#\w+")
NON_LETTER_RE = re.compile(r"[^a-zA-ZÀ-ÿ\s]")


def to_lower(text: str) -> str:
    """Transforma tudo em minúsculo"""
    return text.lower()


def remove_urls(text: str) -> str:
    """Remove URLs do texto"""
    return URL_RE.sub(" ", text)


def remove_mentions(text: str) -> str:
    """Remove @menções (como em redes sociais)"""
    return MENTION_RE.sub(" ", text)


def remove_hashtags(text: str) -> str:
    """Remove #hashtags"""
    return HASHTAG_RE.sub(" ", text)


def remove_non_letters(text: str) -> str:
    """Remove números, emojis e símbolos, mantendo apenas letras e espaços"""
    return NON_LETTER_RE.sub(" ", text)


def basic_clean(text: str) -> str:
    """
    Limpeza básica:
    - minúsculas
    - remove URL, menções, hashtags
    - remove caracteres não alfabéticos
    - retira espaços extras
    """
    if not isinstance(text, str):
        text = str(text)

    text = to_lower(text)
    text = remove_urls(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)
    text = remove_non_letters(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def tokenize(text: str) -> List[str]:
    """Tokenização simples por espaço (após limpeza básica)"""
    text = basic_clean(text)
    if not text:
        return []
    return text.split()


def remove_stopwords(tokens: List[str]) -> List[str]:
    """Remove stopwords em português"""
    return [t for t in tokens if t not in stopwords_pt]


def lemmatize_tokens(tokens: List[str]) -> List[str]:
    """Lematiza tokens usando spaCy (pt_core_news_sm)"""
    if not tokens:
        return []
    doc = nlp(" ".join(tokens))
    lemmas = [tok.lemma_ for tok in doc if tok.lemma_.strip()]
    return lemmas


def stem_tokens(tokens: List[str]) -> List[str]:
    """Aplica stemming usando RSLPStemmer (Português)"""
    if not tokens:
        return []
    return [stemmer.stem(t) for t in tokens]


def preprocess_text(text: str, mode: str = "lemma", remove_stops: bool = True) -> str:
    """
    Pré-processamento completo.
    mode:
      - "lemma": usa lematização
      - "stem": usa stemming
      - None / "none": apenas limpeza básica
    """
    tokens = tokenize(text)

    if remove_stops:
        tokens = remove_stopwords(tokens)

    if mode == "lemma":
        tokens = lemmatize_tokens(tokens)
    elif mode == "stem":
        tokens = stem_tokens(tokens)
    # se mode == "none", não modifica os tokens além da limpeza / stopwords

    return " ".join(tokens)


def preprocess_lemma(text: str) -> str:
    """Atalho: pré-processamento padrão com lematização"""
    return preprocess_text(text, mode="lemma", remove_stops=True)


def preprocess_stem(text: str) -> str:
    """Atalho: pré-processamento com stemming"""
    return preprocess_text(text, mode="stem", remove_stops=True)


def preprocess_no_clean(text: str) -> str:
    """
    Função de 'sem pré-processamento' para comparação:
    apenas garante que o texto é string.
    """
    if not isinstance(text, str):
        return str(text)
    return text


In [7]:
example_text = df["review_text"].iloc[0]
print("Texto original:\n", example_text)
print("\nPré-processado (lemma):\n", preprocess_lemma(example_text))
print("\nPré-processado (stem):\n", preprocess_stem(example_text))


Texto original:
 Estou muito satisfeito, o visor é melhor do que eu imaginava, boas imagens, desing ultra fino. Pelo preço é um exelente aparelho.

O que gostei: Desing exelente, display, custo beneficio.

O que não gostei: Não tem como adicionar mais papeis de parede, bateria dura pouco.

Pré-processado (lemma):
 satisfeito visor melhor imaginar bom imagem desing ultra fino preço exelente aparelho gostar desing exelente display custo beneficio gostar adicionar papel parede bateria durar pouco

Pré-processado (stem):
 satisfeit vis melhor imagin boa imag desing ultr fin preç exel aparelh gost desing exel display cust benefici gost adicion papel pared bat dur pouc


# Questão 2 – Extração e remoção de atributos (`atribute_extraction`)

Nesta questão, são implementadas funções para representar o texto e realizar
remoção/seleção de atributos, utilizando:

- **CountVectorizer** (frequência de termos);
- **TF-IDF** (frequência ponderada);
- **Matriz de coocorrência** (XᵀX, relação entre termos);
- **Word2Vec** (representações distribuídas de palavras);
- **Análise estatística individual** (SelectKBest + teste qui-quadrado).

As funções são organizadas em uma classe `AttributeExtractor`, permitindo reutilizar
o mesmo objeto em diferentes experimentos de classificação.


In [8]:
class AttributeExtractor:
    """
    Classe que organiza diferentes formas de representação / exclusão de atributos:
    - CountVectorizer
    - TF-IDF
    - Matriz de coocorrência
    - Word2Vec (média dos vetores)
    - Seleção estatística (chi^2)
    """

    def __init__(self, max_features: int = 5000, ngram_range: Tuple[int, int] = (1, 1)):
        self.max_features = max_features
        self.ngram_range = ngram_range

        self.count_vect: Optional[CountVectorizer] = None
        self.tfidf_vect: Optional[TfidfVectorizer] = None
        self.cooc_vocab_: Optional[List[str]] = None
        self.cooc_matrix_: Optional[np.ndarray] = None
        self.w2v_model: Optional[Word2Vec] = None
        self.selector_: Optional[SelectKBest] = None

    # ============ COUNT VECTORIZER ============

    def fit_count(self, texts: List[str]) -> csr_matrix:
        self.count_vect = CountVectorizer(
            max_features=self.max_features,
            ngram_range=self.ngram_range
        )
        X = self.count_vect.fit_transform(texts)
        return X

    def transform_count(self, texts: List[str]) -> csr_matrix:
        if self.count_vect is None:
            raise RuntimeError("CountVectorizer não foi ajustado. Chame fit_count primeiro.")
        return self.count_vect.transform(texts)

    # ============ TF-IDF ============

    def fit_tfidf(self, texts: List[str]) -> csr_matrix:
        self.tfidf_vect = TfidfVectorizer(
            max_features=self.max_features,
            ngram_range=self.ngram_range
        )
        X = self.tfidf_vect.fit_transform(texts)
        return X

    def transform_tfidf(self, texts: List[str]) -> csr_matrix:
        if self.tfidf_vect is None:
            raise RuntimeError("TfidfVectorizer não foi ajustado. Chame fit_tfidf primeiro.")
        return self.tfidf_vect.transform(texts)

    # ============ MATRIZ DE COOCORRÊNCIA ============

    def fit_cooccurrence(self, texts: List[str]) -> np.ndarray:
        """
        Usa CountVectorizer para construir a matriz documento-termo X,
        depois calcula uma matriz de coocorrência aproximada: C = X^T X.
        """
        temp_vect = CountVectorizer(
            max_features=self.max_features,
            ngram_range=self.ngram_range
        )
        X = temp_vect.fit_transform(texts)
        vocab = temp_vect.get_feature_names_out()

        cooc = (X.T @ X).toarray().astype(np.float32)
        np.fill_diagonal(cooc, 0.0)

        self.cooc_vocab_ = list(vocab)
        self.cooc_matrix_ = cooc
        return cooc

    def get_cooccurrence(self) -> Tuple[List[str], np.ndarray]:
        if self.cooc_vocab_ is None or self.cooc_matrix_ is None:
            raise RuntimeError("Coocorrência ainda não foi ajustada. Chame fit_cooccurrence primeiro.")
        return self.cooc_vocab_, self.cooc_matrix_

    # ============ WORD2VEC ============

    def fit_word2vec(self, tokenized_texts: List[List[str]],
                     vector_size: int = 100,
                     window: int = 5,
                     min_count: int = 2,
                     workers: int = 4) -> None:
        """
        Treina um modelo Word2Vec nos textos já tokenizados.
        """
        self.w2v_model = Word2Vec(
            sentences=tokenized_texts,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            workers=workers
        )

    def _sentence_to_vec(self, tokens: List[str]) -> np.ndarray:
        """
        Calcula o embedding médio dos tokens de uma sentença.
        """
        if self.w2v_model is None:
            raise RuntimeError("Word2Vec ainda não foi treinado. Chame fit_word2vec primeiro.")

        vectors = []
        for t in tokens:
            if t in self.w2v_model.wv:
                vectors.append(self.w2v_model.wv[t])

        if not vectors:
            return np.zeros(self.w2v_model.vector_size, dtype=np.float32)

        return np.mean(vectors, axis=0)

    def transform_word2vec(self, tokenized_texts: List[List[str]]) -> np.ndarray:
        """
        Gera um vetor para cada sentença (média dos embeddings das palavras).
        """
        if self.w2v_model is None:
            raise RuntimeError("Word2Vec ainda não foi treinado. Chame fit_word2vec primeiro.")

        emb_list = [self._sentence_to_vec(tokens) for tokens in tokenized_texts]
        return np.vstack(emb_list)

    # ============ ANÁLISE ESTATÍSTICA INDIVIDUAL (CHI^2) ============

    def fit_statistical_selector(self, X_train, y_train, k: int = 2000):
        """
        Seleção de atributos via chi^2 (análise estatística individual).
        X_train: matriz esparsa (ex: Count ou TF-IDF)
        y_train: rótulos
        """
        self.selector_ = SelectKBest(chi2, k=k)
        self.selector_.fit(X_train, y_train)

    def transform_with_selector(self, X):
        if self.selector_ is None:
            raise RuntimeError("Selector não ajustado. Chame fit_statistical_selector primeiro.")
        return self.selector_.transform(X)


# Questão 3 – Classificação

Utilizaremos **um único classificador** (Logistic Regression) para comparar:

a) Resultados com **todas as formas de remoção de atributos**, com e sem pré-processamento.  
b) Comparação entre as formas de remoção de atributos, usando o **melhor pré-processamento**.  
c) Comparação entre **lemmatização e stemming**, usando a **melhor forma de remoção de atributos**.

O conjunto é dividido em treino e teste (80/20), estratificando pela `polarity`.
As métricas analisadas aqui serão principalmente as **taxas de acerto (accuracy)**.


In [9]:
def load_X_y_from_df(df: pd.DataFrame,
                     text_col: str = "review_text",
                     label_col: str = "polarity") -> Tuple[pd.Series, pd.Series]:

    df_clean = df[[text_col, label_col]].copy()

    # Remove valores ausentes no texto ou no rótulo
    df_clean = df_clean.dropna(subset=[text_col, label_col])

    # Converte rótulos para inteiro
    df_clean[label_col] = df_clean[label_col].astype(int)

    X_text = df_clean[text_col].astype(str)
    y = df_clean[label_col]

    return X_text, y


def train_and_evaluate(X_train_vec, X_test_vec, y_train, y_test) -> float:
    """
    Treina LogisticRegression e retorna acurácia
    """
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    acc = accuracy_score(y_test, y_pred)
    return acc


def experiment_a(X_text, y):
    """
    (a) Usando todas as formas de remoção de atributos,
    compare resultados com pré-processamento e sem pré-processamento
    """
    results = []

    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_text, y, test_size=0.2, random_state=42, stratify=y
    )

    preproc_options = {
        "sem_preproc": preprocess_no_clean,
        "lemma": preprocess_lemma,
    }

    for preproc_name, preproc_fun in preproc_options.items():
        print(f"=== Pré-processamento: {preproc_name} ===")
        X_train_pp = X_train_raw.apply(preproc_fun)
        X_test_pp = X_test_raw.apply(preproc_fun)

        extractor = AttributeExtractor(max_features=5000, ngram_range=(1, 2))

        # 1) CountVectorizer
        X_train_count = extractor.fit_count(list(X_train_pp))
        X_test_count = extractor.transform_count(list(X_test_pp))
        acc_count = train_and_evaluate(X_train_count, X_test_count, y_train, y_test)
        results.append({
            "preproc": preproc_name,
            "features": "Count",
            "accuracy": acc_count
        })

        # 2) TF-IDF
        X_train_tfidf = extractor.fit_tfidf(list(X_train_pp))
        X_test_tfidf = extractor.transform_tfidf(list(X_test_pp))
        acc_tfidf = train_and_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test)
        results.append({
            "preproc": preproc_name,
            "features": "TF-IDF",
            "accuracy": acc_tfidf
        })

        # 3) TF-IDF + seleção chi²
        extractor.fit_statistical_selector(X_train_tfidf, y_train, k=2000)
        X_train_tfidf_sel = extractor.transform_with_selector(X_train_tfidf)
        X_test_tfidf_sel = extractor.transform_with_selector(X_test_tfidf)
        acc_tfidf_sel = train_and_evaluate(X_train_tfidf_sel, X_test_tfidf_sel, y_train, y_test)
        results.append({
            "preproc": preproc_name,
            "features": "TF-IDF + chi^2",
            "accuracy": acc_tfidf_sel
        })

        # 4) Word2Vec
        tokenized_train = [tokenize(t) for t in X_train_pp]
        tokenized_test = [tokenize(t) for t in X_test_pp]
        extractor.fit_word2vec(tokenized_train, vector_size=100, window=5, min_count=2)
        X_train_w2v = extractor.transform_word2vec(tokenized_train)
        X_test_w2v = extractor.transform_word2vec(tokenized_test)
        acc_w2v = train_and_evaluate(X_train_w2v, X_test_w2v, y_train, y_test)
        results.append({
            "preproc": preproc_name,
            "features": "Word2Vec",
            "accuracy": acc_w2v
        })

    results_df = pd.DataFrame(results)
    print("\n=== Resultados Questão 3(a) ===")
    display(results_df.pivot(index="features", columns="preproc", values="accuracy"))
    return results_df


def experiment_b(X_text, y, best_preproc: str = "lemma"):
    """
    (b) Compare as formas de remoção de atributos usando o pré-processamento escolhido.
    """
    preproc_map = {
        "lemma": preprocess_lemma,
        "sem_preproc": preprocess_no_clean,
        "stem": preprocess_stem,
    }

    preproc_fun = preproc_map[best_preproc]

    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_text, y, test_size=0.2, random_state=42, stratify=y
    )

    X_train_pp = X_train_raw.apply(preproc_fun)
    X_test_pp = X_test_raw.apply(preproc_fun)

    extractor = AttributeExtractor(max_features=5000, ngram_range=(1, 2))
    results = []

    # Count
    X_train_count = extractor.fit_count(list(X_train_pp))
    X_test_count = extractor.transform_count(list(X_test_pp))
    acc_count = train_and_evaluate(X_train_count, X_test_count, y_train, y_test)
    results.append({"features": "Count", "accuracy": acc_count})

    # TF-IDF
    X_train_tfidf = extractor.fit_tfidf(list(X_train_pp))
    X_test_tfidf = extractor.transform_tfidf(list(X_test_pp))
    acc_tfidf = train_and_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test)
    results.append({"features": "TF-IDF", "accuracy": acc_tfidf})

    # TF-IDF + chi²
    extractor.fit_statistical_selector(X_train_tfidf, y_train, k=2000)
    X_train_tfidf_sel = extractor.transform_with_selector(X_train_tfidf)
    X_test_tfidf_sel = extractor.transform_with_selector(X_test_tfidf)
    acc_tfidf_sel = train_and_evaluate(X_train_tfidf_sel, X_test_tfidf_sel, y_train, y_test)
    results.append({"features": "TF-IDF + chi^2", "accuracy": acc_tfidf_sel})

    # Word2Vec
    tokenized_train = [tokenize(t) for t in X_train_pp]
    tokenized_test = [tokenize(t) for t in X_test_pp]
    extractor.fit_word2vec(tokenized_train, vector_size=100, window=5, min_count=2)
    X_train_w2v = extractor.transform_word2vec(tokenized_train)
    X_test_w2v = extractor.transform_word2vec(tokenized_test)
    acc_w2v = train_and_evaluate(X_train_w2v, X_test_w2v, y_train, y_test)
    results.append({"features": "Word2Vec", "accuracy": acc_w2v})

    results_df = pd.DataFrame(results)
    print("\n=== Resultados Questão 3(b) (pré-processamento:", best_preproc, ") ===")
    display(results_df)
    return results_df


def experiment_c(X_text, y, best_features: str = "TF-IDF + chi^2"):
    """
    (c) Compare lematização vs stemming usando a melhor forma de atributos (best_features)
    """
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_text, y, test_size=0.2, random_state=42, stratify=y
    )

    preproc_modes = {
        "lemma": preprocess_lemma,
        "stem": preprocess_stem,
    }

    results = []

    for mode_name, preproc_fun in preproc_modes.items():
        print(f"=== Comparando modo: {mode_name} com features: {best_features} ===")
        X_train_pp = X_train_raw.apply(preproc_fun)
        X_test_pp = X_test_raw.apply(preproc_fun)

        extractor = AttributeExtractor(max_features=5000, ngram_range=(1, 2))

        if best_features == "Count":
            X_train_v = extractor.fit_count(list(X_train_pp))
            X_test_v = extractor.transform_count(list(X_test_pp))

        elif best_features == "TF-IDF":
            X_train_v = extractor.fit_tfidf(list(X_train_pp))
            X_test_v = extractor.transform_tfidf(list(X_test_pp))

        elif best_features == "TF-IDF + chi^2":
            X_train_tfidf = extractor.fit_tfidf(list(X_train_pp))
            X_test_tfidf = extractor.transform_tfidf(list(X_test_pp))
            extractor.fit_statistical_selector(X_train_tfidf, y_train, k=2000)
            X_train_v = extractor.transform_with_selector(X_train_tfidf)
            X_test_v = extractor.transform_with_selector(X_test_tfidf)

        elif best_features == "Word2Vec":
            tokenized_train = [tokenize(t) for t in X_train_pp]
            tokenized_test = [tokenize(t) for t in X_test_pp]
            extractor.fit_word2vec(tokenized_train, vector_size=100, window=5, min_count=2)
            X_train_v = extractor.transform_word2vec(tokenized_train)
            X_test_v = extractor.transform_word2vec(tokenized_test)

        else:
            raise ValueError("best_features inválido")

        acc = train_and_evaluate(X_train_v, X_test_v, y_train, y_test)
        results.append({
            "preproc_mode": mode_name,
            "features": best_features,
            "accuracy": acc
        })

    results_df = pd.DataFrame(results)
    print("\n=== Resultados Questão 3(c) ===")
    display(results_df)
    return results_df


In [11]:
# Carregar X e y a partir do DataFrame
X_text, y = load_X_y_from_df(df)

# Questão 3(a)
res_a = experiment_a(X_text, y)

# >>> Aqui se usa o pré-processamento que foi escolhido no item a)
best_preproc = "lemma"

# Questão 3(b)
res_b = experiment_b(X_text, y, best_preproc=best_preproc)

# >>> Aqui se escolhe a melhor forma de atributos com base em res_b
best_features = "TF-IDF"

# Questão 3(c)
res_c = experiment_c(X_text, y, best_features=best_features)

# Salvar resultados
res_a.to_csv("resultados_q3a.csv", index=False)
res_b.to_csv("resultados_q3b.csv", index=False)
res_c.to_csv("resultados_q3c.csv", index=False)


=== Pré-processamento: sem_preproc ===
=== Pré-processamento: lemma ===

=== Resultados Questão 3(a) ===


preproc,lemma,sem_preproc
features,,
Count,0.943569,0.947644
TF-IDF,0.947440,0.951990
TF-IDF + chi^2,0.947100,0.950428
Word2Vec,0.932093,0.931957



=== Resultados Questão 3(b) (pré-processamento: lemma ) ===


,features,accuracy
0,Count,0.943569
1,TF-IDF,0.947440
2,TF-IDF + chi^2,0.947100
3,Word2Vec,0.932432


=== Comparando modo: lemma com features: TF-IDF ===
=== Comparando modo: stem com features: TF-IDF ===

=== Resultados Questão 3(c) ===


,preproc_mode,features,accuracy
0,lemma,TF-IDF,0.947440
1,stem,TF-IDF,0.948119
